In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
## Read the dataset
training = spark.read.csv('test3.csv',header=True,inferSchema=True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 25|         8| 25000|
|  Sandesh| 28|         4| 20000|
|    Sunny| 24|         3| 20000|
|     Paul| 21|         1| 15000|
|    Harsh| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [9]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols=['Age','Experience'],outputCol='Independent Features')

In [10]:
output = feature_assembler.transform(training)

In [11]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|Age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 25|         8| 25000|          [25.0,8.0]|
|  Sandesh| 28|         4| 20000|          [28.0,4.0]|
|    Sunny| 24|         3| 20000|          [24.0,3.0]|
|     Paul| 21|         1| 15000|          [21.0,1.0]|
|    Harsh| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [12]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [13]:
finalized_data = output.select('Independent Features','Salary')

In [14]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [25.0,8.0]| 25000|
|          [28.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [23]:
from pyspark.ml.regression import LinearRegression
## train test split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [24]:
### Coefficients
regressor.coefficients

DenseVector([218.3509, 1338.4834])

In [25]:
### Intercepts
regressor.intercept

9406.432287561123

In [26]:
### Prediction
pred_results = regressor.evaluate(test_data)

In [33]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [23.0,2.0]| 18000|17105.47059750907|
+--------------------+------+-----------------+



In [30]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(894.5294024909308, 800182.8519207817)